# 📊 Análise de Resultados — PySpark + Delta Lake

Este notebook tem como objetivo realizar análises exploratórias e agregações a partir dos dados estruturados na camada **Trusted** do Data Warehouse, utilizando **PySpark** em conjunto com o **Delta Lake**. Serão exploradas informações provenientes de tabelas fato e dimensões, permitindo insights sobre o perfil das empresas, distribuição geográfica, porte, natureza jurídica, entre outros aspectos relevantes.

## Configurações e importes

In [5]:
# Para iniciar a seção spark
from pyspark.sql import SparkSession, DataFrame
from pyspark.sql.types import StructType, StructField, StringType, DoubleType, DecimalType
from pyspark.sql import functions as f
from delta import configure_spark_with_delta_pip
from delta.tables import DeltaTable

In [6]:
builder = SparkSession.builder \
    .appName("App analise de dados") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config("spark.driver.memory", "2g") \
    .config("spark.executor.memory", "2g")

spark: SparkSession = configure_spark_with_delta_pip(builder).getOrCreate()

spark

## MERGE

Tabelas para merge: 

- cnpj_base & cnpj_basico & empresa_cnpj_basico

In [ ]:
try:    
    deltaTable: DeltaTable = DeltaTable.forPath(spark, "../TRS/CONSOLIDADO/01_TABELA_FATO")
    df: DataFrame = deltaTable.toDF()
    df.show(5, truncate=False)
    df.printSchema()
except Exception as e:
    print(f"Erro na leitura: {e}")

+--------------+---------+----------+-------+-------------+----------------------------------+------------------+-----------------------+-------------------------+--------------------+-----------+---------------------+-----------+----------------------+---------------+--------------------+------+-----------+----------+--------+---+----------------+----+---------+----+---------+-------+----+---------------------------+-----------------+----------------------+-----------+-------------------------------------------------------------------------------+------------+--------------+-------------------+------------+---------------------+-------------------------------+--------------+--------------------+---------------------------+------------------------+-----------------+-----------+-------------+------------------+---------------------+---------+--------------+-----------------+
|cnpj_completo |cnpj_base|cnpj_ordem|cnpj_dv|matriz_filial|nome_fantasia                     |situacao_cadastral|

In [14]:
# Realizando um UPINSERT
try:
    deltaTable.alias("target") \
        .merge(
            df.alias("source"),
            condition="target.cnpj_completo = source.cnpj_completo"
    ) \
        .whenMatchedUpdate(
            set={"cnpj_base": "source.empresa_cnpj_basico"}
    ).whenNotMatchedInsert(
            values={
                "cnpj_completo": "source.cnpj_completo",
                "cnpj_base": "source.empresa_cnpj_basico"
            }
    ).execute()

    print("Merge efetuado!")
except Exception as e:
    print(f"Erro ao efetuar merge: {e}")

Merge efetuado!


In [ ]:
# Filtrar e remover colunas

colunas_filtradas = [
    col for col in df.columns
    if col not in ["empresa_cnpj_basico", "cnpj_basico"]
]

df_novo = df.select(*colunas_filtradas)
df_novo.show()
df_novo.printSchema()

+--------------+---------+----------+-------+-------------+-----------------+------------------+-----------------------+-------------------------+--------------------+-----------+---------------------+-----------+----------------------+---------------+--------------------+------+--------------------+--------------------+--------+---+----------------+----+---------+----+---------+-------+--------+--------------------+-----------------+----------------------+-----------+--------------------+------------+--------------------+--------------------+---------------------+-------------------------------+--------------+--------------------+---------------------------+------------------------+--------------------+-------------+------------------+---------------------+---------+--------------+-----------------+
| cnpj_completo|cnpj_base|cnpj_ordem|cnpj_dv|matriz_filial|    nome_fantasia|situacao_cadastral|data_situacao_cadastral|motivo_situacao_cadastral|nome_cidade_exterior|codigo_pais|data_inic

In [ ]:
try:
    df_novo.write.format("delta") \
        .mode("overwrite") \
        .option("overwriteSchema", "true") \
        .save("../CONSOLIDADO/01_TABELA_FATO")
except Exception as e:
    print(f"Erro ao gravar novos dados: {e}")